In [15]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [16]:
# Load Data
data_dir = './data/'
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
               "hot_dog", "lightning", "mountain", "skull",
               "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [17]:
X = []
y = []

# Append data one by one from each file in data folder to X and y
for label, class_name in enumerate(class_names):
    file_path = os.path.join(data_dir, f"{class_name}.npy")
    data = np.load(file_path)  # shape: (30000, 784)

    X.append(data)
    y.append(np.full((data.shape[0],), label))

In [19]:
np.array(X).shape

(19, 30000, 784)

In [20]:
np.array(y).shape

(19, 30000)

In [21]:

X = np.vstack(X)  # shape: (570000, 784)
y = np.hstack(y)  # shape: (570000,)

In [ ]:
# ✅ Max Normalization for image pixel data
X = X.astype('float32') / 255.0  # max value of a pixel is 255

In [25]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)

In [27]:
# ✅ Improved and Simplified Model
model = Sequential([
    Input(shape=(784,)),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(19, activation='linear')  # logits
])

In [28]:
# Compile
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

## Callbacks

A callback is a function/object that runs at specific stages of model training — like at the end of each epoch, or when accuracy stops improving.

1. `EarlyStopping(patience=5, restore_best_weights=True)` Monitors validation loss (by default).

    If validation loss doesn't improve for 5 epochs (`patience=5`), it stops training early.

    It also restores the best model weights seen during training if `restore_best_weights=True`.
    

2. `ReduceLROnPlateau(patience=3, factor=0.5)` Monitors validation loss (or another metric).

    If the metric doesn’t improve for 3 epochs, it reduces the learning rate by 50%.

In [29]:

# ✅ Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [32]:

# ✅ Increased batch size for stability
model.fit(
    X_train, y_train,
    validation_data=(X_cv, y_cv),
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.8494 - loss: 0.6541 - val_accuracy: 0.8833 - val_loss: 0.5303 - learning_rate: 0.0010
Epoch 2/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8669 - loss: 0.5920 - val_accuracy: 0.8931 - val_loss: 0.4985 - learning_rate: 0.0010
Epoch 3/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8761 - loss: 0.5574 - val_accuracy: 0.8961 - val_loss: 0.4828 - learning_rate: 0.0010
Epoch 4/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8816 - loss: 0.5374 - val_accuracy: 0.9006 - val_loss: 0.4693 - learning_rate: 0.0010
Epoch 5/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8868 - loss: 0.5186 - val_accuracy: 0.9024 - val_loss: 0.4630 - learning_rate: 0.0010
Epoch 6/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8880 - loss: 0.5137 - val_accuracy: 0.9042 - val_loss: 0.4585 - learning_rate: 0.0010
Epoch 7/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8

In [33]:
# Evaluation
logits_cv = model.predict(X_cv)
y_cv_pred = tf.argmax(tf.nn.softmax(logits_cv, axis=1), axis=1).numpy()
print("Validation Accuracy:", accuracy_score(y_cv, y_cv_pred))

logits_test = model.predict(X_test)
y_test_pred = tf.argmax(tf.nn.softmax(logits_test, axis=1), axis=1).numpy()
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

1782/1782 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
Validation Accuracy: 0.9296491228070175
1782/1782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Test Accuracy: 0.9313508771929825


In [34]:
# Save model and normalization
model.save("doodle_model.keras")